In [1]:
import numpy as np
import pandas as pd

import itertools
import time
from datetime import datetime

from sklearn.cross_validation import KFold
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn import preprocessing
from sklearn.cluster import KMeans
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import make_scorer, mean_absolute_error

from scipy.stats import skew, boxcox
from scipy.sparse import csr_matrix, hstack, spmatrix

import xgboost as xgb

/home/w/anaconda2/envs/idp2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
xgb_tr1 = pd.read_csv('Good_preds/XGB_LGBM/train2_LGBM_30.11_10folds_0.003eta_interactions.csv')
xgb_tr2 = pd.read_csv('Good_preds/XGB_LGBM/train2_XGBM_2&3_30.11_10folds_0.01eta_interactions.csv')
xgb_tr3 = pd.read_csv('Good_preds/XGB_LGBM/train3_preds_XGB2&3+LGBM3_interactions_eta0.03.csv')
xgb_tr4 = pd.read_csv('Good_preds/XGB_LGBM/train1_XGB4_eta0.01_2.12.csv')
xgb_tr5 = pd.read_csv('Good_preds/XGB_LGBM/train3_preds_no_interactions_eta0.01_oldparams.csv')
xgb_tr6 = pd.read_csv('Good_preds/XGB_LGBM/train1_XGBL2_stack_log+210_4.12.csv')
xgb_tr7 = pd.read_csv('Good_preds/XGB_LGBM/train1_XGB_Alikernel.csv')['0']

nn_tr1 = pd.read_csv('Good_preds/NN/train1_kernelbagging_log+210.csv')
nn_tr2 = pd.read_csv('Good_preds/NN/train1_nn3_10folds_10bags_loss+1.csv')
nn_tr3 = pd.read_csv('Good_preds/NN/train1_kernelbagging_nonlog.csv')
nn_tr4 = pd.read_csv('Good_preds/NN/train1_nn3_10fold5bags_log+210.csv')
nn_tr5 = pd.read_csv('Good_preds/NN/train1_nn3_10folds_10bags_loss+1.csv')
nn_tr6 = pd.read_csv('Good_preds/NN/train1_nn4_10folds_10bags_loss+1.csv')
nn_tr7 = pd.read_csv('Good_preds/NN/train1_nn1_log+210.csv')
nn_tr8 = pd.read_csv('Good_preds/NN/train1_NNL2_raw_10folds3bags_6.12.csv')

sklearn_knn = pd.read_csv('Good_preds/Sklearn/train_knn_sklearn_algos_4.12.csv')
sklearn_sparse = pd.read_csv('Good_preds/Sklearn/train_sparse_sklearn_algos_5.12.csv')
etrf_tr = pd.read_csv('Good_preds/Sklearn/train2_ET&RF_Faron_6.12.csv')


nn_tr1.set_index('id', inplace = True)
nn_tr1.sort_index(inplace = True)
nn_tr1.reset_index(inplace = True)

nn_tr3.set_index('id', inplace = True)
nn_tr3.sort_index(inplace = True)
nn_tr3.reset_index(inplace = True)

nn_tr1 = nn_tr1['loss']
nn_tr3 = nn_tr3['loss']

preds_tr = pd.concat([xgb_tr1, xgb_tr2, xgb_tr3, xgb_tr4, xgb_tr5, xgb_tr6, xgb_tr7,
                      nn_tr1, nn_tr3, nn_tr2, nn_tr4, nn_tr5, nn_tr6, nn_tr7, nn_tr8,
                     sklearn_knn, sklearn_sparse, etrf_tr], 1)
preds_tr.columns = xrange(preds_tr.shape[1])

X_train1 = (preds_tr + 1) ** 0.25
y_train1 = (pd.read_csv('../../train.csv')['loss'] + 1) ** 0.25



In [3]:
X_train2 = X_train1.iloc[:, [1, 3, 6, 7, 11, 20, 22, 25, 26, 33, 34]]

In [10]:
def gbmr_model(num_leaves, min_data_in_leaf, feature_fraction, bagging_fraction, max_depth):
    
    gbmr = GBMRegressor(
        exec_path='/path/to/your/LightGBM/lightgbm', # Change this to your LighGBM path 
        config='',
        application='regression',
        num_iterations = 20000,
        learning_rate=0.01,
        num_leaves=int(num_leaves),
        tree_learner='serial',
        num_threads = -1,
        min_data_in_leaf=int(min_data_in_leaf),
        max_depth = int(max_depth),
        metric='l1',
        feature_fraction=max(feature_fraction,0),
        feature_fraction_seed = 42,
        bagging_fraction=max(bagging_fraction,0),
        bagging_freq=100,
        bagging_seed = 42,
        metric_freq=1,
        early_stopping_round = 50,
        verbose=False,
    )
    
    return gbmr


def gbmr_foldeval(num_leaves, min_data_in_leaf, feature_fraction, bagging_fraction, max_depth):

    t = time.time()
    scores = []
    best_rounds = []
    
    gbmr2 = gbmr_model(num_leaves, min_data_in_leaf, feature_fraction, bagging_fraction, max_depth)
    
    for i, (train_index, test_index) in enumerate(kf):
            print "Training on fold no. : {}".format(i + 1)

            X_train = X_train1.values[train_index]
            y_train = y_train1[train_index]
            X_val = X_train1.values[test_index]
            y_val = y_train1[test_index]
            pred = np.zeros(X_val.shape[0])

            gbmr2.fit(X_train, y_train, test_data = [(X_val, y_val)])
            preds = gbmr2.predict(X_val)

            score = mean_absolute_error((y_val ** 4) - 1, (preds ** 4) - 1)
            scores.append(score)
            best_round = gbmr2.best_round
            best_rounds.append(best_round)

            print "Validation score for round {}".format(i + 1), score
            print "Best round score for round {}".format(i + 1), gbmr2.best_round

    mean_score = np.array(scores).mean()
    mean_best_round = int(np.mean(best_rounds))
    print "Mean score: ", mean_score
    print "Averaged best round: ", mean_best_round
    print "Time it took: ", time.time() - t 

    return -mean_score

In [11]:
from bayes_opt import BayesianOptimization
from pylightgbm.models import GBMRegressor

   1 | 05m00s | -1123.57836 |             0.9769 |             0.3039 |      5.1240 |            61.3959 |     114.8604 |

In [12]:
kf = KFold(y_train1.shape[0], n_folds = 5, shuffle = True, random_state = 111)

ntrain = y_train1.shape[0]
num_iter = 10
init_points = 10

gbmrBO = BayesianOptimization(gbmr_foldeval, 
                              {
                                'num_leaves': (100, 200),
                                'min_data_in_leaf': (2, 100),
                                'max_depth': (5, 15),
                                'feature_fraction': (0.2, 0.9),
                                'bagging_fraction': (0.7, 1),
                              }
                             )

gbmrBO.maximize(init_points=init_points, n_iter=num_iter)

print('Final Results')
print('LightGBM: %f' % gbmrBO.res['max']['max_val'])

Initialization
-------------------------------------------------------------------------------------------------------------------------
 Step |   Time |      Value |   bagging_fraction |   feature_fraction |   max_depth |   min_data_in_leaf |   num_leaves | 
Training on fold no. : 1
Validation score for round 1 1112.51027386
Best round score for round 1 541
Training on fold no. : 2
Validation score for round 2 1121.26139728
Best round score for round 2 527
Training on fold no. : 3
Validation score for round 3 1134.87732925
Best round score for round 3 554
Training on fold no. : 4
Validation score for round 4 1123.16853075
Best round score for round 4 547
Training on fold no. : 5
Validation score for round 5 1126.07425146
Best round score for round 5 569
Mean score:  1123.57835652
Averaged best round:  547
Time it took:  300.897793055
    1 | 05m00s | -1123.57836 |             0.9769 |             0.3039 |      5.1240 |            61.3959 |     114.8604 | 
Training on fold no. : 1
Vali

KeyboardInterrupt: 

In [ ]:

def output(name_train, name_test, files_train, files_test):
    
    train_preds = pd.DataFrame(np.concatenate(files_train, axis = 1))
    test_preds = pd.DataFrame(np.concatenate(files_test, axis = 1))
    
    train_preds.to_csv('Good_preds/XGB_LGBM/{}.csv'.format(name_train), index = False)
    test_preds.to_csv('Good_preds/XGB_LGBM/{}.csv'.format(name_test), index = False)
                      
    return


In [ ]:
ids = pd.read_csv('../../test.csv')['id']
submission_file = "{}/sample_submission.csv".format(directory)

submission = pd.read_csv(submission_file)
submission.iloc[:, 1] = xgb_te11
submission.to_csv('Good_preds/XGBL2_stack_loss+1_4.12.csv', index = False)